In [209]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.svm import LinearSVC
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
post_train = pd.read_csv("../../data/howpop_train.csv")
post_test = pd.read_csv("../../data/howpop_test.csv")

In [4]:
post_test.head() 

,url,domain,post_id,published,author,flow,polling,content_len,title
0,https://habrahabr.ru/post/314080/,habrahabr.ru,314080,2016-11-01 01:05:00,@fsou11,develop,True,20132,Опыт использования MassTransit 3.0
1,https://habrahabr.ru/company/plesk/blog/313732/,habrahabr.ru,313732,2016-11-01 05:42:00,NaN,marketing,False,12389,Геймификация форума на движке XenForo
2,https://habrahabr.ru/company/etagi/blog/314000/,habrahabr.ru,314000,2016-11-01 08:33:00,NaN,admin,False,72839,Кластер высокой доступности на postgresql 9.6 ...
3,https://habrahabr.ru/company/knopka/blog/314030/,habrahabr.ru,314030,2016-11-01 09:00:00,NaN,develop,False,29572,Как перестать бояться и полюбить синтаксически...
4,https://geektimes.ru/company/audiomania/blog/2...,geektimes.ru,282058,2016-11-01 09:44:00,NaN,NaN,False,9642,Мифы и реальность: Что нужно знать о современн...


In [5]:
Post_title_train = pd.DataFrame(post_train.drop(['post_id', 'url', 'domain', 'author', 'flow', 'polling',
                                    'content_len', 'comments', 'favs', 'views',
                                    'votes_plus', 'votes_minus', 'views_lognorm',
                                    'favs_lognorm', 'comments_lognorm'], axis=1))

Post_title_test = pd.DataFrame(post_test.drop(['post_id', 'url', 'domain', 'author', 'flow', 'polling',
                                    'content_len'], axis=1))                 

In [6]:
Post_target_train =   pd.DataFrame(post_train.drop(['post_id', 'url', 'domain', 'published','author', 'flow', 'polling', 'title',
                                    'content_len', 'comments', 'views',
                                    'votes_plus', 'votes_minus', 'views_lognorm',
                                    'favs_lognorm', 'comments_lognorm'], axis=1))

In [7]:
cv = CountVectorizer()
cv.fit(Post_title_train['title'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [8]:
print(len(cv.vocabulary_))

100202


In [9]:
print(cv.get_feature_names()[:50])

['00', '000', '00000000', '000008', '0001', '000webhost', '000р', '001', '0010', '002', '002x', '003', '004', '005', '007', '009', '00x', '01', '010', '0101', '01100100', '012', '013', '01410', '0183', '01e', '02', '020', '0204', '021', '0224', '023', '0249', '02b', '03', '030', '034', '036', '04', '0402', '045', '0497', '05', '0507', '051', '057', '06', '060', '0603', '061']


In [10]:
print(cv.get_feature_names()[50000:50050])

['киберугроз', 'киберугрозам', 'киберугрозы', 'киберуслуг', 'киберхелпер', 'кибершпионаж', 'кибершпионажа', 'кибершпионажем', 'кибершпионское', 'киберэпоху', 'кибитка', 'кибкало', 'киборг', 'киборга', 'киборгам', 'киборгами', 'киборги', 'киборгизация', 'киборгов', 'киборифе', 'киви', 'кид', 'кидаем', 'кидай', 'кидалова', 'киддисов', 'кидков', 'кидок', 'кидхак', 'киев', 'киева', 'киеве', 'киевская', 'киевские', 'киевский', 'киевских', 'киевского', 'киевской', 'киевском', 'киевстар', 'кикстартер', 'кикстартера', 'кикстартере', 'кикстартеру', 'киллер', 'килобайт', 'килобайта', 'килобайтный', 'киловатт', 'киловаттную']


In [172]:
X_train = cv.transform(Post_title_train['title'])
X_test = cv.transform(Post_title_test['title'])

### Добавлю новые признаки

In [12]:
new_feat_train = pd.DataFrame(index=Post_title_train.index)
new_feat_test = pd.DataFrame(index=Post_title_test.index)

In [188]:
new_feat_train['year_m'] = Post_title_train['published'].apply(lambda ts: pd.to_datetime(ts))
new_feat_test['year_m'] = Post_title_test['published'].apply(lambda ts: pd.to_datetime(ts))

In [189]:
new_feat_train['year_m'] = new_feat_train['year_m'].apply(lambda ts: ts.year * 100 + ts.month)
new_feat_test['year_m'] = new_feat_test['year_m'].apply(lambda ts: ts.year * 100 + ts.month)

Заскейлю данные

In [196]:
scaler = StandardScaler()
scaler.fit(new_feat_train['year_m'].values.reshape(-1, 1))
# Создадим новый признак который уже будет маштабированым
new_feat_train['year_m_scaled'] = scaler.transform(new_feat_train['year_m'].values.reshape(-1, 1))
new_feat_test['year_m_scaled'] = scaler.transform(new_feat_test['year_m'].values.reshape(-1, 1))

In [212]:
def get_auc_lr_valid(X, y, A = 0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    y_valid = y_valid.astype(float)
    
    ridge = Ridge(alpha=A)
    ridge.fit(X_train, y_train)
    val = ridge.predict(X_valid)
    #logit.predict - будет предсказывать 0 или 1. Ответы будут некоректны и иногда неправельны. Поэтому лучше предсказывать вероятности
    #valid_pred = pd.Series(np.reshape(val, len(val)), index=y_valid.index) # [:, 1] - берём только второй столбец где предсказанавероятность что это 1
    return np.sqrt(mean_squared_error(y_valid, val)), r2_score(y_valid, val)

In [216]:
X_train_new = csr_matrix(hstack([X_train, new_feat_train['year_m_scaled'].values.reshape(-1, 1)]))
X_test_new = csr_matrix(hstack([X_test, new_feat_test['year_m_scaled'].values.reshape(-1, 1)]))

array([1.        , 1.        , 1.        , ..., 1.        , 1.        ,
       1.41774972])

In [220]:
get_auc_lr_valid(X_train_new, Post_target_train, 1,  0.7)

(161.27213345052303, -0.845792329567574)